In [102]:
import numpy as np
import scipy.linalg as linalg
from dtuimldmtools import similarity
import matplotlib.pyplot as plt
from scipy.linalg import svd
from pandas.plotting import scatter_matrix
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pyplot import figure, legend, plot, show, xlabel, ylabel
# exercise 8.1.1
from dtuimldmtools import dbplotf, train_neural_net, visualize_decision_boundary

import importlib_resources
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pylab import (
    figure,
    grid,
    legend,
    loglog,
    semilogx,
    show,
    subplot,
    title,
    xlabel,
    ylabel,
)
from scipy.io import loadmat
from sklearn import model_selection

from dtuimldmtools import rlr_validate
import numpy as np
import scipy.linalg as linalg
from dtuimldmtools import similarity
import matplotlib.pyplot as plt
from scipy.linalg import svd
from pandas.plotting import scatter_matrix
import pandas as pd
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pyplot import figure, legend, plot, show, xlabel, ylabel
# exercise 8.1.1
from dtuimldmtools import dbplotf, train_neural_net, visualize_decision_boundary
import importlib_resources
import numpy as np
import torch
from matplotlib.pyplot import figure, show, title
from scipy.io import loadmat
import importlib_resources
import numpy as np
import sklearn.linear_model as lm
from matplotlib.pylab import (
    figure,
    grid,
    legend,
    loglog,
    semilogx,
    show,
    subplot,
    title,
    xlabel,
    ylabel,
)
from scipy.io import loadmat
from sklearn import model_selection

from dtuimldmtools import rlr_validate

# fetch dataset 
# wine = fetch_ucirepo(id=109) 
  
# # data (as pandas dataframes) 
# X = wine.data.features 
# y = wine.data.targets 

# totaldata= (wine.data)
  
# metadata 
# print(wine.metadata) 
  
# variable information 
# print(wine.variables) 

# OFFLINE LOADING OF DATA
X = np.loadtxt('../wine/wine.data', delimiter=',')

#Extract classes from X
y = X[:,0]
X = np.delete(X,0,axis=1)
# y = np.loadtxt('../wine/wine.names', delimiter=',')

Xorig = X
# Standardizing the data

X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)



attributeNames = [
    "Alcohol",
    "Malic acid",
    "Ash",
    "Alcalinity of ash",
    "Magnesium",
    "Total phenols",
    "Flavanoids",
    "Nonflavanoid phenols",
    "Proanthocyanins",
    "Color intensity",
    "Hue",
    "OD280/OD315 of diluted wines",
    "Proline"
]

N, M = X.shape

# Add offset attribute
attributeNames = ["Offset"] + attributeNames
# M = M + 1


# Convert class labels to 0, 1, 2
y = y - 1

classNames = ["1", "2", "3"]


In [122]:
# %% Model fitting and prediction using logistic regression

## Crossvalidation
# Create OUTER crossvalidation partition for evaluation
K = 2
CV = model_selection.KFold(K, shuffle=True, random_state=42)
# CV = model_selection.KFold(K, shuffle=False)

# Values of lambda
lambdas = np.logspace(0, 3, 100)
# Initialize variables
# T = len(lambdas)
Error_train = np.empty((K, 1))
Error_test = np.empty((K, 1))
Error_train_rlr = np.empty((K, 1))
Error_test_rlr = np.empty((K, 1))
Error_train_nofeatures = np.empty((K, 1))
Error_test_nofeatures = np.empty((K, 1))
w_rlr = np.empty((M, K))
mu = np.empty((K, M - 1))
sigma = np.empty((K, M - 1))
w_noreg = np.empty((M, K))

k = 0
for train_index, test_index in CV.split(X, y):
    k += 1
    Error_train_avg = []
    Error_test_avg = []
    # extract training and test set for current CV fold
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    internal_cross_validation = 2
    
    internal_cv = model_selection.KFold(internal_cross_validation, shuffle=True, random_state=42)

    # Inner cross-validation to find the best regularization parameter (lambda)
    best_lambda = 0.0
    min_error = float('inf')
    for train_inner_idx, val_inner_idx in internal_cv.split(X_train, y_train):
        print("Outer Fold: ", k)
        #FOR LOGISTIC REGRESSION
        for l in lambdas:
            Error_train_rlr_fold = []
            Error_test_rlr_fold = []
            error_inner = []
        # For each lambda we do the training with internal cross validation
            X_train_inner, y_train_inner = X_train[train_inner_idx], y_train[train_inner_idx]
            
            X_val_inner, y_val_inner = X_train[val_inner_idx], y_train[val_inner_idx]

            logreg = lm.LogisticRegression(C=1/l, solver="lbfgs", tol=1e-4, random_state=1)
            logreg.fit(X_train_inner, y_train_inner)
            
            error_val = np.mean(logreg.predict(X_val_inner) != y_val_inner)

        # If the error is less than the minimum error, we update the best lambda
            # if error_val < min_error:
            #     min_error = error_val
            #     best_lambda = l
                
            error_train_new = np.mean(logreg.predict(X_train_inner) != y_train_inner)
            Error_train_rlr_fold.append(error_train_new)
            Error_test_rlr_fold.append(error_val)
            
        #_____________________________________________________
        # FOR ANN
        # Define the model structure
        max_hidden_units = 3
        error_log = []
        for n_hidden_units in range(1, max_hidden_units):
            print("Number of hidden units: ", n_hidden_units)
            # number of hidden units in the signle hidden layer
            C = 3
            model = lambda: torch.nn.Sequential(
                torch.nn.Linear(M, n_hidden_units),  # M features to H hiden units
                torch.nn.ReLU(),  # 1st transfer function
                # Output layer:
                # H hidden units to C classes
                # the nodes and their activation before the transfer
                # function is often referred to as logits/logit output
                torch.nn.Linear(n_hidden_units, C),  # C logits
                # To obtain normalised "probabilities" of each class
                # we use the softmax-funtion along the "class" dimension
                # (i.e. not the dimension describing observations)
                torch.nn.Softmax(dim=1),  # final tranfer function, normalisation of logit output
            )
            # Since we're training a multiclass problem, we cannot use binary cross entropy,
            # but instead use the general cross entropy loss:
            loss_fn = torch.nn.CrossEntropyLoss()
            # Train the network:
            # C = 3
            net, _, _ = train_neural_net(
                model,
                loss_fn,
                X=torch.tensor(X_train_inner, dtype=torch.float),
                y=torch.tensor(y_train_inner, dtype=torch.long),
                n_replicates=3,
                max_iter=3000
            )
            # Determine probability of each class using trained network
            softmax_logits = net(torch.tensor(X_val_inner, dtype=torch.float))
            # Get the estimated class as the class with highest probability (argmax on softmax_logits)
            y_test_est = (torch.max(softmax_logits, dim=1)[1]).data.numpy()
            # Determine errors
            e = y_test_est != y_val_inner
            # print(
            #     "Number of miss-classifications for ANN:\n\t {0} out of {1}".format(sum(e), len(e))
            # )
            error_log.append(np.mean(e))

            predict = lambda x: (
                torch.max(net(torch.tensor(x, dtype=torch.float)), dim=1)[1]
            ).data.numpy()
        #Find the best amount of hidden units
        # print(error_log)
        # print(range(1, max_hidden_units))
        # print(np.argmin(error_log))
        best_hidden_unit = range(1, max_hidden_units)[np.argmin(error_log)]
    
    #For ANN We just train with the best 
    #_____________________________________________________
    # Average errors across folds for each hidden unit
    final_error_for_ANN = []
    C = 3
    model = lambda: torch.nn.Sequential(
        torch.nn.Linear(M, best_hidden_unit),  # M features to H hiden units
        torch.nn.ReLU(),  # 1st transfer function
        # Output layer:
        # H hidden units to C classes
        # the nodes and their activation before the transfer
        # function is often referred to as logits/logit output
        torch.nn.Linear(best_hidden_unit, C),  # C logits
        # To obtain normalised "probabilities" of each class
        # we use the softmax-funtion along the "class" dimension
        # (i.e. not the dimension describing observations)
        torch.nn.Softmax(dim=1),  # final tranfer function, normalisation of logit output
    )
    # Since we're training a multiclass problem, we cannot use binary cross entropy,
    # but instead use the general cross entropy loss:
    loss_fn = torch.nn.CrossEntropyLoss()
    # Train the network:
    # C = 3
    net, _, _ = train_neural_net(
        model,
        loss_fn,
        X=torch.tensor(X_train, dtype=torch.float),
        y=torch.tensor(y_train, dtype=torch.long),
        n_replicates=3,
        max_iter=3000
    )
    # Determine probability of each class using trained network
    softmax_logits = net(torch.tensor(X_test, dtype=torch.float))
    # Get the estimated class as the class with highest probability (argmax on softmax_logits)
    y_test_est = (torch.max(softmax_logits, dim=1)[1]).data.numpy()
    # Determine errors
    print(
        "Number of miss-classifications for ANN:\n\t {0} out of {1}".format(sum(final_error_for_ANN), len(final_error_for_ANN))
    )
    
    predict = lambda x: (
        torch.max(net(torch.tensor(x, dtype=torch.float)), dim=1)[1]
    ).data.numpy()
    
    final_error_for_ANN.append( np.mean(y_test_est != y_test))
    print("Best hidden unit: ", best_hidden_unit)
    print("final ANN error for k=" + str(k) + " is " + str(np.mean(y_test_est != y_test)))
    
    
    
    #FOR LAMBDA / LOGISTIC REGRESSION
    #_____________________________________________________
    # Average errors across folds for each lambda
    Error_test_avg.append(np.mean(Error_test_rlr_fold))
    
    best_lambda = lambdas[np.argmin(Error_test_avg)]
    
    
    print("the best lambda is: ", best_lambda)
    
    
    # Train with best lambda on outer fold and evaluate amount of errors
    logreg_best = lm.LogisticRegression(C=best_lambda, solver="lbfgs",  tol=1e-4, random_state=1)
    logreg_best.fit(X_train, y_train)

    # Multinomial logistic regression
    logreg = lm.LogisticRegression(
        C=best_lambda,solver="lbfgs", multi_class="multinomial", tol=1e-4, random_state=1
    )
    logreg.fit(X_train, y_train)
    final_error_for_logreg = []
    final_error_for_logreg.append(np.mean(logreg.predict(X_test) != y_test))
    print("final logreg error for k=" + str(k) + " is " + str(np.mean(logreg.predict(X_test) != y_test)))

    #_____________________________________________________
    
    # To display coefficients use print(logreg.coef_). For a 4 class problem with a
    # feature space, these weights will have shape (4, 2).

    # Number of miss-classifications
    # print(
    #     "Number of miss-classifications for Multinormal regression:\n\t {0} out of {1}".format(
    #         np.sum(logreg.predict(X_test) != y_test), len(y_test)
    #     )
    # )
    # print(logreg.predict(X_test))
    # print(y_test)
    
    # print("Coefficients")
    # print(logreg.coef_)
    # # Plotting should be done after the outer loop
    # plt.figure(figsize=(10, 6))
    # plt.semilogx(lambdas, Error_train_avg, 'b-', label='Training Error')
    # plt.semilogx(lambdas, Error_test_avg, 'r-', label='Test Error')
    # plt.xlabel('Lambda')
    # plt.ylabel('Error')
    # plt.legend()
    # plt.title('Training vs Test Error for Different Lambda Values')
    # plt.grid()
    # plt.show()
    # k += 1
# print("final_error_for_ANN: ", (final_error_for_ANN))
# print("final_error_for_logreg: ", (final_error_for_logreg))


Outer Fold:  1
Number of hidden units:  1

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.8102478	5.722911e-05
		2000	0.77741617	3.7106995e-05
		3000	0.7543922	1.9752148e-05
		Final loss:
		3000	0.7543922	1.9752148e-05

	Replicate: 2/3
		Iter	Loss			Rel. loss


/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	0.8730904	0.00022509911
		2000	0.7585879	8.265213e-05
		3000	0.71848965	3.591963e-05
		Final loss:
		3000	0.71848965	3.591963e-05

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.8908395	2.4220253e-05
		2000	0.8761322	7.448901e-05
		3000	0.8075554	4.0150342e-05
		Final loss:
		3000	0.8075554	4.0150342e-05
Number of hidden units:  2

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.8245678	6.77996e-05
		2000	0.5938711	0.00014049308
		3000	0.5642925	1.9329407e-05
		Final loss:
		3000	0.5642925	1.9329407e-05

	Replicate: 2/3
		Iter	Loss			Rel. loss
		1000	0.6279545	0.0001843931
		2000	0.57166487	3.4406272e-05
		3000	0.56045616	1.0847606e-05
		Final loss:
		3000	0.56045616	1.0847606e-05

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.6860005	0.00023106647
		2000	0.6065466	5.3946696e-05
		3000	0.58788455	1.7235723e-05
		Final loss:
		3000	0.58788455	1.7235723e-05
Outer Fold:  1
Number of hidden units:  1

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.805317	7.193633e-05
		2000	

/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	0.9001961	0.00014538251
		2000	0.77596146	6.436592e-05
		3000	0.7333832	4.5348577e-05
		Final loss:
		3000	0.7333832	4.5348577e-05

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.8895106	8.877819e-05
		2000	0.86223686	7.46576e-06
		3000	0.8566322	3.6181577e-06
		Final loss:
		3000	0.8566322	3.6181577e-06
Number of hidden units:  2

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.6133242	0.00021200819
		2000	0.5655703	2.3395714e-05
		3000	0.55726624	8.128819e-06
		Final loss:
		3000	0.55726624	8.128819e-06

	Replicate: 2/3
		Iter	Loss			Rel. loss
		1000	0.6018061	0.00013428419
		2000	0.56730086	2.5845833e-05
		3000	0.55875343	8.960563e-06
		Final loss:
		3000	0.55875343	8.960563e-06

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.6196018	0.00019889853
		2000	0.57094944	3.2570388e-05
		3000	0.56016296	1.0853284e-05
		Final loss:
		3000	0.56016296	1.0853284e-05

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.61805135	0.00019689095
		2000	0.5687199	3.0916493e-05
		3000	0.5587

...

/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	0.84059805	6.048038e-05
		Final loss:
		1982	0.81268656	2.93371e-07

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.889577	4.1540356e-05
		2000	0.850976	1.7510365e-05
		3000	0.8079183	2.397649e-05
		Final loss:
		3000	0.8079183	2.397649e-05
Number of hidden units:  2

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.81461585	0.00014207406
		2000	0.6274368	0.00013373795
		3000	0.6044309	1.1340348e-05
		Final loss:
		3000	0.6044309	1.1340348e-05

	Replicate: 2/3
		Iter	Loss			Rel. loss
		1000	0.6380181	0.00027598452
		2000	0.58827597	2.2999313e-05
		3000	0.58046454	7.495904e-06
		Final loss:
		3000	0.58046454	7.495904e-06

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.64580786	0.00016333495
		2000	0.5965367	3.5069923e-05
		3000	0.58417314	1.193766e-05
		Final loss:
		3000	0.58417314	1.193766e-05
Outer Fold:  2
Number of hidden units:  1

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.75080144	6.477643e-05
		2000	0.72687614	1.7711924e-05
		3000	0.69222933	0.00010099137
		Fin

/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/dtuimldmtools/models/nn_trainer.py:141: RuntimeWarning: overflow encountered in cast
  if loss_value < best_final_loss:


		1000	0.7722469	9.315169e-05
		2000	0.74115413	1.5923182e-05
		3000	0.7344081	5.0319018e-06
		Final loss:
		3000	0.7344081	5.0319018e-06

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.759604	7.2656134e-05
		2000	0.7318719	2.3617436e-05
		3000	0.6775463	0.00012077003
		Final loss:
		3000	0.6775463	0.00012077003
Number of hidden units:  2

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.69913906	0.0011701942
		2000	0.5675323	2.8460758e-05
		3000	0.558467	8.858432e-06
		Final loss:
		3000	0.558467	8.858432e-06

	Replicate: 2/3
		Iter	Loss			Rel. loss
		1000	0.6516074	0.00015740067
		2000	0.57700497	5.12342e-05
		3000	0.5612145	1.41252685e-05
		Final loss:
		3000	0.5612145	1.41252685e-05

	Replicate: 3/3
		Iter	Loss			Rel. loss
		1000	0.601477	0.00019181524
		2000	0.5645633	2.3331873e-05
		3000	0.5571114	7.489156e-06
		Final loss:
		3000	0.5571114	7.489156e-06

	Replicate: 1/3
		Iter	Loss			Rel. loss
		1000	0.6326692	0.00021560337
		2000	0.57260126	3.8721657e-05
		3000	0.5601756	1.202

/Users/lauge/Desktop/ML/MLenv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
